# Parte 3 - Machine Learning Workflow

Datasets: [Diamanti](https://www.kaggle.com/shivam2503/diamonds)

**OBBIETTVO:** In base alle sue caratteristiche provare a predire il prezzo di un diamante <br>
Utilizzeremo la libreria python **scikit-learn** per testare alcuni algoritmi di classificiazione!

In [52]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import svm, preprocessing

diamond_df = pd.read_csv("datasets/diamonds.csv", index_col = 0)

In [53]:
diamond_df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


**Quale modello algoritmo di classificazione dovremmo utilizzare?**

[Come scegliere il corretto algoritmo](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

In [54]:
len(diamond_df)

53940

## Linear Regression

<img src="images/regression_1.jpeg" alt="Drawing" style="width: 545px;"/><img src="images/regression_2.jpeg" alt="Drawing" style="width: 500px;"/> <br> Per utilizzare la Linear Regression deve esserci una relazione lineare tra i dati

<img src="images/regression_3.png" alt="Drawing" style="width: 545px;"/> 

[Least Square Method](https://www.varsitytutors.com/hotmath/hotmath_help/topics/line-of-best-fit)

Come si definisce la best fit line? <br>

$$Y = mx + b$$

Data una X dobbiamo trovare la sua Y corrispondente, ma prima dobbiamo risolvere _m_ e _b_:

_m_ è la pendenza<br>
_b_ è l'intersezione di y

$$m = \frac{\overline{x}\cdot\overline{y} - \overline{xy}}{(\overline{x})^2-\overline{x^2}}$$

$$b = \overline{y}-m\overline{x} $$

In questo caso abbiamo dati su 2 dimensioni, ma appena incrementiamo la dimensione dello spazio verriale incrementerà anche la complessità dei calcoli

**Per allenare il nostro modello vogliamo utilizzare tutti i paramentri tranne il prezzo**

In [55]:
diamond_df["cut"].unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

Abbiamo bisogno di categorie numeriche!

In [56]:
diamond_df["cut"].astype("category").cat.codes

1        2
2        3
3        1
4        3
5        1
        ..
53936    2
53937    1
53938    4
53939    3
53940    2
Length: 53940, dtype: int8

**Problema**: Dobbiamo preservare il significato delle labels: per esempio Premium sarà migliore di Fair e così via..

In [57]:
cut_dizionario = {"Fair":1, "Good":2, "Very Good":3, "Premium":4, "Ideal":5}

Stessa cosa per:

In [58]:
clarity_dizionario = {"I3": 1, "I2": 2, "I1": 3, "SI2": 4, "SI1": 5, "VS2": 6, "VS1": 7, "VVS2": 8, "VVS1": 9, "IF": 10, "FL": 11}
color_dizionario = {"J": 1,"I": 2,"H": 3,"G": 4,"F": 5,"E": 6,"D": 7}

Bisognerà mappare queste classi alle varie colonne del dataset

In [59]:
diamond_df['cut'] = diamond_df['cut'].map(cut_dizionario)

## Esercizio 13:
- Mappare le colonne "clarity" e "color" con i rispettivi dizionari!

In [60]:
#Esercizio
diamond_df['clarity'] = diamond_df['clarity'].map(clarity_dizionario)
diamond_df['color'] = diamond_df['color'].map(color_dizionario)

In [61]:
diamond_df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,5,6,4,61.5,55.0,326,3.95,3.98,2.43
2,0.21,4,6,5,59.8,61.0,326,3.89,3.84,2.31
3,0.23,2,6,7,56.9,65.0,327,4.05,4.07,2.31
4,0.29,4,2,6,62.4,58.0,334,4.20,4.23,2.63
5,0.31,2,1,4,63.3,58.0,335,4.34,4.35,2.75


Prima di allenare il nostro modello è importante mescolare i dati per evitare che si formi del biasing analizzando i dati in ordine.
<br> Per esempio potrebbe essere, come nel nostro caso, che i dati siano ordinati per prezzo.

In [62]:
diamond_df

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,5,6,4,61.5,55.0,326,3.95,3.98,2.43
2,0.21,4,6,5,59.8,61.0,326,3.89,3.84,2.31
3,0.23,2,6,7,56.9,65.0,327,4.05,4.07,2.31
4,0.29,4,2,6,62.4,58.0,334,4.20,4.23,2.63
5,0.31,2,1,4,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,5,7,5,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,2,7,5,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,3,7,5,62.8,60.0,2757,5.66,5.68,3.56
53939,0.86,4,3,4,61.0,58.0,2757,6.15,6.12,3.74


In [63]:
diamond_df = sklearn.utils.shuffle(diamond_df)

Separiamo il set di features dalla label che dobbiamo predire

In [74]:
X = diamond_df.drop("price", axis=1).values #Feature Set --> Ogni label tranne quella che dobbiamo predire
y = diamond_df["price"].values

In [75]:
X

array([[2.08, 5.  , 4.  , ..., 8.22, 8.18, 5.07],
       [0.4 , 3.  , 4.  , ..., 4.73, 4.7 , 2.98],
       [1.31, 4.  , 2.  , ..., 6.97, 6.93, 4.38],
       ...,
       [0.95, 1.  , 1.  , ..., 6.17, 6.12, 3.99],
       [0.6 , 5.  , 5.  , ..., 5.42, 5.45, 3.34],
       [0.7 , 1.  , 7.  , ..., 5.45, 5.52, 3.62]])

**Bonus**: Avremmo potuto barare caricando il dataframe con l'index, poiché il dataset era ordinato per prezzo e facendo ciò avremmo lasciato un informazione in più che avrebbe compromesso la nostra regressione, poiché l'indice sarebbe stato ordinato come il prezzo.

**Preprocessing**: Permette di normalizzare i valori, in questo modo ridurremo la sparsità e il modello lavorerà con dati più uniformi con conseguente miglioramente nelle performance

In [76]:
print(np.mean(X))

16.596651670580478


In [77]:
X = preprocessing.scale(X) #Cerca

In [78]:
print(np.mean(X))

3.2311501327895136e-16


In [37]:
test_size = 200

Il **train** è una porzione del dataset per cui il modello viene allenato

Il **test** è una porzione del dataset che il nostro modello non vedrà mai, e sarà usato per valutarne le performance.

In [79]:
X_train = X[:-test_size]
y_train = y[:-test_size]

X_test = X[-test_size:]
y_test = y[-test_size:]

Nel caso utilizzassimo il dataset di test per allenare il modello l'accurattezza finale risulterebbe compromessa poiché i dati sono già stati visionati dal modello durante la fase di train, in questo modo non possiamo verificare tramite il test se il modello abbia realmente imparato a predire un valore o abbia solamente imparato a memoria il dataset di train. Per questo motivo i dati di test devono essere utilizzati solo per testare il modello!

Andiamo a selezionare il [modello](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) da utilizzare

In [81]:
clf = svm.SVR(kernel="linear")
clf.fit(X_train, y_train)

SVR(kernel='linear')

In [82]:
clf.score(X_test, y_test)

0.8976448088812292

Coefficiente di determinazione (R quadro): 
- 0.0 Caso peggiore 
- 1.0 Caso migliore

Esso è calcolato partendo dall'errore quadratico medio

Fin troppo bello per essere vero!

Andiamo a verificare quello che è successo!

In [83]:
for X,y in zip(X_test, y_test):
    print(f"Model: {clf.predict([X])}, Actual: {y}")
    

Model: [3301.56887613], Actual: 3093
Model: [700.87921669], Actual: 881
Model: [13492.10926782], Actual: 13200
Model: [11927.23385104], Actual: 14130
Model: [7986.77641652], Actual: 8033
Model: [1629.20475456], Actual: 1577
Model: [916.03632614], Actual: 990
Model: [5220.65694161], Actual: 6479
Model: [589.94200934], Actual: 827
Model: [451.14632834], Actual: 743
Model: [2008.18138527], Actual: 1439
Model: [960.63746592], Actual: 842
Model: [38.32809791], Actual: 556
Model: [31.05473824], Actual: 462
Model: [1013.57947654], Actual: 912
Model: [5021.55835055], Actual: 4421
Model: [3614.38740382], Actual: 3087
Model: [7235.3752142], Actual: 10138
Model: [3119.52704782], Actual: 2753
Model: [6185.271093], Actual: 6363
Model: [4564.51016651], Actual: 4876
Model: [8506.82097249], Actual: 8777
Model: [5536.51476394], Actual: 7412
Model: [211.44741257], Actual: 720
Model: [690.44529001], Actual: 965
Model: [1202.86922743], Actual: 982
Model: [4824.9576056], Actual: 5292
Model: [5408.58031777]

Testiamo un altro modello!

In [84]:
clf = svm.SVR(kernel="rbf")
clf.fit(X_train, y_train)

SVR()

In [85]:
clf.score(X_test, y_test)

0.6407873072161727

In [86]:
for X,y in zip(X_test, y_test):
    print(f"Model: {clf.predict([X])}, Actual: {y}")

Model: [3283.35443205], Actual: 3093
Model: [1212.49381085], Actual: 881
Model: [5503.97079347], Actual: 13200
Model: [6185.42992763], Actual: 14130
Model: [6564.64363937], Actual: 8033
Model: [1710.75887773], Actual: 1577
Model: [1080.27450262], Actual: 990
Model: [5071.05628882], Actual: 6479
Model: [1109.77898257], Actual: 827
Model: [858.22636889], Actual: 743
Model: [1544.31621247], Actual: 1439
Model: [884.26799853], Actual: 842
Model: [722.61033461], Actual: 556
Model: [1061.53734915], Actual: 462
Model: [991.86497341], Actual: 912
Model: [4917.38718935], Actual: 4421
Model: [3398.79108293], Actual: 3087
Model: [6270.97536075], Actual: 10138
Model: [2902.12187111], Actual: 2753
Model: [6403.72046795], Actual: 6363
Model: [4657.19793438], Actual: 4876
Model: [6927.68021207], Actual: 8777
Model: [5821.59914859], Actual: 7412
Model: [591.91335578], Actual: 720
Model: [1121.1681425], Actual: 965
Model: [1055.89830077], Actual: 982
Model: [5074.25158961], Actual: 5292
Model: [4628.20

**BONUS:** Si potrebbero usare più classifier e alla fine fare una media delle prestazioni!